# Non-Additive Polygenic model

We assume that genotypes are stored in a $N \times L$ matrix, where $N$ is the number of individuals and $L$ is the number of loci. The model has the following parameters:

$s$, vector of effect sizes for different loci

$e$, power term that controls linearity

$a^2$, variance of the Gaussian error term (which has mean $0$)
$b^2$, variance increase of the Gaussian error term (as commonly seen in Poisson-liked distributions)

$\mu$, population-wide mean

We define the mean for individual $i$ as

$$\w_i = sgn(b_i)|b_i|^\alpha + \mu,$$
where $b_i = \beta^T X_i = \sum_j \beta_j X_{ij}$. Alternatively, we write the phenotype $Y_i$ for individual $i$ as 

$$Y_i = sgn(b_i)|b_i|^\alpha + \mu + \epsilon_i,$$
where $\epsilon_i$ is the Gaussian error for individual $i$.

In this model, the loglikelihood is
\begin{align*}
LL(\theta) &= \sum_{i=1}^n \log P(Y_i \mid \theta) \\
&= \sum_{i=1}^n \log \frac{1}{\sqrt{2\pi \sigma^2}} \exp\left(-\frac{(Y_i-\mu_i)^2}{2\sigma^2}\right)\\
&= -\frac{n}{2}\log 2 \pi \sigma^2 - \frac{1}{2\sigma^2}\sum_{i=1}^n \left(Y_i-\mu_i\right)^2.
\end{align*}

### gradients

First, the gradient for the effect size $\beta_l$. Defining operator $D := \frac{\delta}{\delta \beta_l}$,

\begin{align*}
DLL(\theta) &= D\left(-n\log \sqrt{2 \pi \sigma^2} - \frac{1}{2\sigma^2}\sum_{i=1}^n \left(Y_i-\mu_i\right)^2\right)\\
&= -\frac{1}{2\sigma ^2}\sum_{i=1}^n D(Y_i-\mu_i)^2 \\
&= \frac{1}{2\sigma^2} \sum_{i=1}^n 2(Y_i-\mu_i)D\mu_i
\end{align*}

We have

\begin{align*}
D\mu_i &= D\left[ sgn(b_i) |b_i|^\alpha  \right] \\
\end{align*}
where $b = \sum_{j=1}^k \beta_j G_{ij}$ and $sgn(x)$ is the sign function. We have $D sgn(b_i) = 0$, so, by the product rule,

\begin{align*}
D \mu_i &= sgn(b) D\left(|b_i|^\alpha\right) \\
&= sgn(b_i)\alpha |b_i|^{\alpha-1} D|b_i|\\
&= sgn(b_i)\alpha |b_i|^{\alpha-1}sgn(b_i)G_{il}\\
&= \alpha |b_i|^{\alpha-1} G_{il}
\end{align*}


And thus
\begin{align*}
&= DLL(\theta) = \frac{\alpha}{\sigma^2} \sum_{i=1}^n\left(Y_i-\mu_i\right)\left|\sum_{j=1}^k \beta_j G_{ij}\right|^{\alpha-1} \cdot G_{il}.
\end{align*}

### Now we need to calculate with respect to $\mu$, $\sigma^2$ and $\alpha$.

#### For $\alpha$

Defining $D:= \frac{\delta}{\delta \alpha}$, we again have

\begin{align}
DLL(\theta) &= \frac{1}{2\sigma^2}\sum_{i=1}^n2(Y_i-\mu_i)D\mu_i \\
\end{align}

We can calculate 

\begin{align*}
D\mu_i &= D\big(sgn(b)|b|^\alpha + \mu\big)\\
&= sgn(b)|b|^\alpha \log|b|
\end{align*}
where again $b = \sum_{j=1}^k \beta_j G_{ij}$.  (And all the log's are base $e$.) Thus

$$\frac{\delta}{\delta \alpha}LL(\theta) = \frac{1}{\sigma^2}\sum_{i=1}^n (Y_i-\mu_i)sgn(b_i)|b_i|^\alpha \log |b|.$$


#### For $\mu$

Defining $D:= \frac{\delta}{\delta \mu}$, we again have

\begin{align}
DLL(\theta) &= \frac{1}{2\sigma^2}\sum_{i=1}^n2(X_i-\mu_i)D\mu_i \\
\end{align}

We can calculate 

\begin{align*}
D\mu_i &= D\big(sgn(b)|b|^\alpha + \mu\big)\\
&= 1
\end{align*}

and thus

$$\frac{\delta}{\delta \mu} LL(\theta) = \frac{1}{\sigma^2}\sum_{i=1}^n(X_i-\mu_i)$$.

#### For $\sigma^2$  (remembering to ignore the squared, since it's just part of the parameter's name)

Defining $D:= \frac{\delta}{\delta \sigma^2}$, we again have

\begin{align}
DLL(\theta) &= D\left(-n\log \sqrt{2 \pi \sigma^2} - \frac{1}{2\sigma^2}\sum_{i=1}^n \left(X_i-\mu_i\right)^2\right)\\
&= -\frac{n}{2\sigma^2} + \frac{1}{2\sigma^2}\sum_{i=1}^n\left(X_i-\mu_i\right)^2
\end{align}

# checking the gradient numerically

In [69]:
import numpy as np
from scipy.optimize import approx_fprime
import numpy.random as npr

In [89]:
k = 10
G = npr.choice([-1,0,1], size = k, replace = True)
beta = npr.uniform(-1,1, size = k)
mu_0 = 4.5
alpha = 4

In [90]:
def mu_i(betas):
    b = np.sum(betas*G)
    absb = np.abs(b)
    sgnb = np.sign(b)
    return sgnb * absb**alpha + mu_0

In [91]:
def Dmu_i(betas):
    absb = np.abs(np.sum(betas*G))
    return alpha*(absb**(alpha-1.0))*G

In [92]:
mu_i(beta)

-8.932148169982433

In [93]:
approx_fprime(beta, mu_i, 1e-9)

array([  0.        , -28.06526389,  28.06526389, -28.06526389,
       -28.06526389,  28.06526389,  28.06526389, -28.06526389,
       -28.06526389,  28.06526389])

In [94]:
Dmu_i(beta)

array([  0.        , -28.06526173,  28.06526173, -28.06526173,
       -28.06526173,  28.06526173,  28.06526173, -28.06526173,
       -28.06526173,  28.06526173])

Checks out!